 <font size="6">   Universidade de São Paulo -  USP/São Carlos
    
    Instituto de Ciências Matemáticas e de Computação - ICMC
    
    Estatística e Ciência de Dados
    
    Grupo: Flaviane Louzeiro da Silva - nUSP 11885075
    
    SCC0652 - Visualização Computacional 
    
    Projeto Prático
 
</font>


# Objetivo Geral

Neste trabalho prático pretende-se explorar a base de dados CORD-19,  afim de clusterizar os artigos. Para isso os termos mais frequentes, as nacionalidades mais citadas, a similaridade dos documentos com TF-IDF e por fim o agrupamento de documentos com o <i>k-means</i>. Dessa forma, este trabalho é divido em, pelo menos, 03 etapas.

<b> Etapa 01 - Organizando o dataframe </b>

Inicialmente será apresentada a base de dados e a estrutura original, em seguida é feito o tratamento da base de dados, realizando a validação da estrutura dos artigos analisados, organizando-os em um <i>dataframe</i>, retirando os artigos com atributos faltosos e os artigos duplicados.

<b> Etapa 02 - Exploração do dataframe e calculo de similaridade dos documentos </b>

Afim de completar a etapa anterior, será realizada a tokinização dos termos, em seguida a sumarização dos dados tratados, com os termos mais frequentes e a nuvem de palavras. Em seguinda será realizada a busca pelas entidades nomeadas, utilizando recursos de processamento de linguagem natural.

<b> Etapa 03 - Agrupamento dos documentos</b>

[Em desenvolvimento...] 


Abaixo segue os links para cada etapa.

[Etapa 01](#etapa1)

[Etapa 02](#etapa2)

[Etapa 03](#etapa3)


# Etapa 01 <a id='etapa1'></a>

## Descrição da base de dados <a id='descricao'></a>

Com o objetivo de ajudar pesquisadores e corpo médico no combate ao Covid-19, foi lançado um desafio Kaggle[1] para tratamento e busca de informações em uma base de dados formada por diversos artigos científicos relacionados ao corona vírus, o **COVID-19 Open Research Dataset (CORD-19)**.

  Atualmente, o CORD-19 possue mais de 200.000 artigos, destes pelos menos 100.000 são artigos completos, gerando em torno de 20GB. No entanto, para esta tarefa usei uma base com apenas 4GB, que contém mais de 29.000 artigos, onde pelo menos 13.000 desses são artigos completos. Esses artigos estão formatados na estrutura *.JSON*

[1] https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge

[Etapa 02] (#etapa_02)

## Visualização da estrutura do dataset <a id='visualizacao'></a>
  
   O dataset CORD-19 possuem estrutura apresentada abaixo, com 4 diretórios principais. 
   <img src="figs/estruturaCORD-19.png"/>
   
   Dentro de cada diretório há os artigos em formato *JSON*. 
   <img src="figs/arquivosJSON.png" />
   
   Os artigo *.JSON* possuem estão estruturados no formato abaixo:
   <img src="figs/estrutruraJSON.png"/>
   
   Onde o título se apresenta conforme a figura a seguir
   <img src="figs/title.png"/>
   
   O resumo está contido na seguinte estrutura.
   <img src="figs/abstract.png"/>
   
   E por fim, o corpo do artigo encontra-se no formato abaixo, onde cada subseção *1,2...* contém um paragrafo da publicação.
   <img src="figs/corpo.png"/>
   
   
   Esta exploração da estrutura do arquivo é necessária pois os códigos desenvolvidos terá que alimentar um dataframe com as informações contidas em cada seção dos artigos contidos no *CORD-19*.

## Metodologia

Conforme visto em aula, a organização e seleção dos dados é a primeiro etapa do pipeline de visualização. Dessa forma a persistência dos dados é crucial nesse processo. 

Em resumo, nesta etapa será realizada a:
* inserção dos artigos em um dataframe;
* exclusão dos artigos que por ventura estejam faltando alguma seção como ID, título, resumo, corpo do texto;
* remoção de artigos duplicados

A seguir será apresentada a metodologia utilizada para manipulação do dataset, onde o objetivo é organizar o dataset em um dataframe para utilizá-lo posteriormente. 

### Instalação e carregamento das Bibliotecas

In [1]:
import pandas as pd
import glob
import json 
import seaborn as sns

### Definição da estrutura do dataframe
Inicialmente será criado um dicionário vazio, que será utilizado para buscar na base de dados os seguintes atributos nos artigos:
* a identificação (paper_id);
* o título (title)
* o resumo (abstract)
* o corpo (text)

In [2]:
dicArtigos = {'paper_id':[], 'title':[],
             'abstract':[], 'text':[]}
type(dicArtigos)

dict

Posteriormente o dicionário é transformado em dataframe, para facilitar a manipulação no decorrer do processamento.

In [3]:
dfArtigos = pd.DataFrame.from_dict(dicArtigos)
type(dfArtigos)

pandas.core.frame.DataFrame

### Listagem dos artigos contidos no dataset
Criada a estrutura, será feita uma busca recursiva pelos arquivos .json presentes no diretório com o auxílio da blibioteca de leitura de diretorio glob. Os arquivos *.json* serão lidos e o nome do arquivo será inserido na lista.

In [4]:
listaArtigos = glob.glob(f'{"./"}//**/*.json', recursive= True) 

print('Foram encontrados ',len(listaArtigos), ' artigos') # total de artigos percorrido

Foram encontrados  29315  artigos


### Carregando o dataframe
A seguir será criada a função que de fato alimentará o dataframe com os atributos definidos anteriormente.

In [5]:
## funcao que vai percorrer todos os aquivos e popular o DF
def funcao_dfArtigos(listaArtigos, df):
    for nomeArtigo in listaArtigos:
        linhaDF={'paper_id':None, 'title':None,
                 'abstract':None, 'text':None}
        
        with open(nomeArtigo) as json_data:
            
            dados = json.load(json_data)
            
            linhaDF['paper_id']=dados['paper_id'].strip() #retirando os espaços no inicio e no fim
            linhaDF['title']=dados['metadata']['title'] 
            # extração palavra por palavra do abstract
            listaAbstract = [abstract['text'] for abstract in dados['abstract']]
            abstract = '\n'.join(listaAbstract)
            linhaDF['abstract']=abstract.strip()
            # extração palavra por palavra do conteudo
            listaTexto = [text['text'] for text in dados['body_text']] 
            texto = '\n'.join(listaTexto)
            linhaDF['text']=texto.strip()
            
            df = df.append(linhaDF, ignore_index = True)
        
    return df
    

Chamada da função. Agora de fato ocorrerá a extração dos dados e insersão no dataframe.

**Observação: Este procedimento poderá demorar alguns minutos.**


In [ ]:
dfArtigos = funcao_dfArtigos(listaArtigos, dfArtigos) 

In [ ]:
print('O dataframe foi alimentado com ',dfArtigos.shape[0], ' artigos e ', dfArtigos.shape[1], ' atributos')

In [ ]:
## salvando o total de artigos para comparar
totalArtigos=dfArtigos.shape[0] 

## Verificação da Base de dados

Inicialmente será verificado no dataframe se há valores faltantes, ou seja, se no dataframe os artigos foram  alimentado com todos os atributos definidos no dicionário. 

Então, por se tratar de uma base de dados textual será será feito um mapa de calor com o auxilio da biblioteca *seaborn*. A intenção é visualmente saber em quais artigos há atributos faltantes. 

In [ ]:
sns.heatmap(dfArtigos.isnull());

**Observação:** De acordo com o mapa de calor, em todos os artigos foram carregadas alguma informação em cada atributo, seja informação útil ou apenas um espaço. No entanto, conforme é possível notar no head do dataframe, há artigos sem título ou sem abstract.

In [ ]:
dfArtigos.head()

#### Verificação dos atributos

Dessa forma será verificado em todas as colunas do dataframe se há algum atributo com apenas um espaço, ou seja ''

In [ ]:
# verificando se há artigo sem ID
print ('Foram encontrados ',len(dfArtigos[dfArtigos['paper_id']=='']), 'artigos sem o ID')

In [ ]:
print ('Foram encontrados ',len(dfArtigos[dfArtigos['title']=='']),'artigos sem o título')

In [ ]:
print ('Foram encontrados ',len(dfArtigos[dfArtigos['abstract']=='']), 'artigos sem o resumo')

In [ ]:
print ('Foram encontrados ',len(dfArtigos[dfArtigos['text']=='']),'artigos sem conteúdo')

#### Remoção dos artigos  
Feita as verificações, será excluído da base de dados os artigos com informações faltantes

In [ ]:
## apagando os artigos que faltam informações
dfArtigos = dfArtigos[dfArtigos['title'] != ''] # deixando somente os artigos que tem titulo

In [ ]:
dfArtigos = dfArtigos[dfArtigos['abstract'] != ''] # deixando somente os artigos que tem abstract

In [ ]:
print('Haviam ', totalArtigos,'após a limpeza ficaram, ', dfArtigos.shape[0],'artigos')

#### Removendo os artigos duplicados
Por fim, usando a função do pandas *drop_duplicates*, será removido os artigos que estejam duplicados no dataframe.

In [ ]:
# remoção de artigos duplicados, o inplace = true remove e atualiza o df
dfArtigos.drop_duplicates(['abstract','text','title'], inplace=True)

In [ ]:
dfArtigos.shape

In [ ]:
print('Haviam ', totalArtigos, 'artigos, após todas as verificações restaram ', dfArtigos.shape[0],'artigos')
print('Uma redução de',round(100-dfArtigos.shape[0]*100/totalArtigos), '% da base de dados')

## Salvando o dataframe

Salvando o dataframe para o envio no Edisciplinas

In [ ]:
dfArtigos.to_csv('dfArtigos_completo.csv')

## Conclusão

O tratamento da base de dados foi o primeiro passo dado. A base de dados é textual e, conforme apresentado anteriormente, houve uma redução de 31% após a verificação da base de dados. Com isso, verificou-se a importância da realização da persistência dos dados, visto que, embora o desafio siga regras e possivelmente tenha seguido uma política controle na inserção dos artigos no dataset, houveram artigos com seções faltosas além de artigos duplicados.

# Etapa 02 <a id='etapa2'></a>

## Objetivo 

Identificar o grupo taxonômico dos dados, apresentar e avaliar o mapeamento visual apropriado. No entanto, para isso é necessário concluir a etapa anterior. Dessa forma, será dada continuidade no tratamento, realizando a tokinização e sumarização dos dados, apresentando-os  em uma nuvem de palavras e uma tabela com os termos mais frequentes. Para isso, além pré processamento já realizado, será necessário inicialmente tratar os dados com bibliotecas de processamento natural 
    

## Amostragem da base

Embora já tenha ocorrido uma redução de 31% dos artigos após o pré processamento. Ainda há muitos artigos, o que resultou em um travamento de minha máquina. Dessa forma, daqui em diante, será trabalhado apenas com uma amostra do dataset, essa amostra foi retirada aleatoriamente da base de dados com 500 artigos. E, para que o processo possa ser repetido, será fixado o random_state = 123.

In [ ]:
dfArtigos_amostra = dfArtigos.sample(n=500, random_state=123)

In [ ]:
dfArtigos_amostra.shape # checando a estrutura do dataframe

In [ ]:
dfArtigos_amostra.head() # checando os 5 primeiros

### Uso da biblioteca científica

Por se tratar de uma base de dados com artigos científicos, o inglês é a língua padrão e acredita-se que há uma baixa taxa de erros, visto que seguem regras bem explícitas pois são  publicados em revista.

Será utilizada a biblioteca **spacy**, que é uma biblioteca para processamento avançado de linguagem natural, sendo bastante utilizada para analisar textos. Nela há o modelo **en_core_sci_md** já treinado na para o jargão médico. 

Dentre as funcionalidades do modelo há uma lista de stop words, que é uma lista de termos não representativos para um documento, geralmente essa lista é composta por: preposições, artigos, advérbios, números, pronomes e pontuação.

In [ ]:
#### instalação das bibliotecas - Se necessário
#pip install spacy
#pip install scispacy
#pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_md-0.2.5.tar.gz

In [ ]:
import spacy
import scispacy
import en_core_sci_md #fazer download na cell acima

A seguir, será criado o modelo, onde serão desabilitados alguns parametros para facilitar o processamento. Além disso será definido o tamanho máximo de 2000000 caracteres a serem analisados em cada artigo.

In [ ]:
modelo = en_core_sci_md.load(disable=['tagger', 'parser', 'ner'])
modelo.max_lenght = 2000000

### Extração e limpeza dos termos nos Textos

Para a maioria dos tipos de processamento lingüístico é necessário identificar e categorizar as palavras de um texto, para isso é realizada a extração e limpeza dos termos, onde cada documento vai ter o seu conteúdo dividido em cada palavra significantiva presente no documento.

Primeiro  é realizada a tokenização para decompor o documento em cada termo que o compõe, delimitando o espaço em branco entre os termos, quebras de linhas, tabulações, e alguns caracteres especiais. 

Além disso, é feita a limpeza, onde são removidos as stop words, depois é apliacada a lematização  que é o método para redução de um termo ao seu radical, removendo as desinências, afixos, e vogais temáticas, dessa forma os termos derivados de um mesmo radical serão contabilizados como um único termo.

In [ ]:
def tokinizacao(sentenca):
    sentenca = sentenca.lower() # colocando miusculo
    
    #lematização 
    lista = []
    lista = [palavra.lemma_ for palavra in modelo(sentenca) if not (palavra.is_stop      #removendo as stop_words e
                                                                   or palavra.like_num   #os numeros
                                                                   or palavra.is_punct   # pontuacao
                                                                   or palavra.is_space   # espaços excedentes 
                                                                   or len(palavra)== 1   # com tamanho 1 caractere                                                       
                                                                   
                                                                
                                                                   )] 
    
    lista = ' '.join([str(palavra) for palavra in lista]) #concatenando as palavras
    
    
    return lista


Testando a função no primeiro artigo da amostra

In [ ]:
teste = dfArtigos_amostra['text'][10258] 
print('Antes da tokinizaçao: ', len(teste), ' palavras')
print('Após a tokinização: ', len(tokinizacao(teste)),' palavras')
print('Uma redução de ', round(len(tokinizacao(teste))/len(teste)*100,2), '%')

Testada a função, agora será aplicada em todos os atributos 'text' dos artigos do dataframe

In [ ]:
dfArtigos_amostra['text'] = dfArtigos_amostra['text'].apply(tokinizacao)

## Sumarização 

### Termos frequentes - Tabela de Frequencia
Será usada a biblioteca NLTK, no entanto ela precisa que as entradas estejam em txt, assim, inicialmente será criado um txt para cada artigo.

In [ ]:
for index, row in dfArtigos_amostra.iterrows(): #percorendo as linhas do DF
    artigoTXT = open('corpus/'+row['paper_id']+'.txt', 'w') # criando o .txt
    n = artigoTXT.write(row['text'])# excrevendo no .txt
    artigoTXT.close()

criando o Corpus

In [ ]:
import nltk
from nltk.corpus import PlaintextCorpusReader
corpus = PlaintextCorpusReader('corpus', '.*')

In [ ]:
n = corpus.words()
print('Há ',len(n), ' palavras no corpus')

In [ ]:
frequenciaPalavra = nltk.FreqDist(n) # distribuição de frequencia
maisFrequentes = frequenciaPalavra.most_common(100) # 100 mais comuns
maisFrequentes

### Termos Frequentes - Nuvem de Palavras

In [ ]:
#pip install wordcloud
from wordcloud import WordCloud
from matplotlib.colors import ListedColormap
from matplotlib import pyplot as plt

In [ ]:
coresMapa = ListedColormap(['orange', 'green', 'red','magenta']) # cores do mapa
nuvemPalavras = WordCloud(background_color = 'white', max_words= 100, colormap = coresMapa)
nuvemPalavras = nuvemPalavras.generate(dfArtigos_amostra['text'].str.cat(sep='\n'))
plt.figure(figsize = (15,15))
plt.imshow(nuvemPalavras)
plt.axis('off') # sem eixo x e y
plt.show()

## Insight - Extração de entidades nomeadas

Utilizando recursos de processamento de linguagem natural, através da biblioteca spacy, será realizada a contagem dos países e nacionalidades mais citadas na amostra, fazendo busca pelas entidades nomeadas do tipo [GPE](https://spacy.io/api/annotation), ou seja, os paises, cidades e nacionalidades mais citadas na amostra. 

Para isso, será criando um modelo de reconhecimento de entidade na língua inglesa, com tamanho máximo de 2000000 caracteres.



In [ ]:
nlp_entidade = spacy.load('en')
nlp_entidade.max_length = 2000000

In [ ]:
gpe = []
for index, row in dfArtigos_amostra.iterrows():
    text = row['text']
    doc = nlp_entidade(text)
    for entidade in doc.ents:
        if entidade.label_ == 'GPE':
            gpe.append(str(entidade.text))
    

Exibindo as entidades com mais de 50 citadas

In [ ]:
import numpy as np
gpe_valor, gpe_qtd = np.unique(np.array(gpe), return_counts = True)
dfGPE = pd.DataFrame({'valor':gpe_valor, 'qtd': gpe_qtd})
dfGPE_filtrado = dfGPE[dfGPE.qtd>50] #  mais de 50 citações
dfGPE_filtrado = dfGPE_filtrado.sort_values(by='qtd', ascending=False)

Barplot das 10 entidades nomeadas mais citadas

In [ ]:
sns.set(rc={'figure.figsize':(15,8)})
sns.barplot(x= 'valor', y='qtd', hue='valor', data = dfGPE_filtrado[:10]);

Com seus dados pré-processados, identifique em qual grupo taxonômico seu conjunto de dados se encaixa (p.e. categórico, numérico, híbrido, etc...). Justifique com uma explicação dos seus dados.

        Vide o catálogo From Data to Viz

Identifique qual mapeamento visual é o mais indicado para os seus dados e, se você julgar que aquele paradigma visual realmente é a melhor escolha, apresente seus dados com a visualização

        Caso decida utilizar outra visualização, justifique a escolha

Descreva os insights que a visualização proporcionou para os seus dados

Para as próximas etapas:

    Definir modelo de representação dos documentos

    Definir objetivos do que você deseja realizar no conjunto de dados

Já existem vários sistemas que trabalham com exploração visual deste conjunto de dados: CORD-19. Pode usar estes como inspiração sobre o que explorar no conjunto de dados

In [ ]:
dfArtigos = pd.read_csv('corona_df_completo.csv')
dfArtigos = dfArtigos.dropna()
dfArtigos.shape


In [ ]:
dfArtigos.to_csv('dfArtigos_completo.csv')

### TD-IDF  

Para pontuar a importância de uma palavra em um documento com base na frequência com que ela apareceu nesse documento.

In [ ]:
textosArtigos = dfArtigos['text'].tolist()

from sklearn.feature_extraction.text import TfidfVectorizer 
tfdif = TfidfVectorizer(max_features=2**12) # para evitar estouro de memoria
artigosVet = tfdif.fit_transform(textosArtigos)

Redução de dimensionalidade com PCA (<i>Principal Componet Analysis </i>) para vizualizar a dispersão dos artigos

In [ ]:
from sklearn.decomposition import PCA
pca = PCA (n_components=2) # 2 componentes
x_pca = pca.fit_transform(artigosVet.toarray())

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.scatterplot(x_pca[:,0],x_pca[:,1])
plt.title('Artigos Covid');

# Etapa 03 <a id='etapa3'></a>

Agrupamento com k-means